# Programming R from Haskell

This notebook demonstrates integrating both R and Haskell in the same notebook, using `IHaskell-inline-r`.

## Prelude

First, a bit of setup. We need to enable quasiquotation, to embed R code into Haskell expressions:

In [ ]:
:ext QuasiQuotes

Next, we need to make sure that R is initialized properly:

In [ ]:
import qualified H.Prelude as H
H.initialize H.defaultConfig

## R computations

By default, computations in code cells are interpreted as Haskell code. For instance, here is a definition of the factorial function, in Haskell:

In [3]:
fact 0 = 1
fact n = n * fact (n - 1)

Here is a Haskell expression calling `fact`, together with its value:

In [4]:
fact 10

3628800

`inline-r` allows you to embed R expressions and R statements anywhere in Haskell code, using quasiquotation. The following is an IO action that asks R to print the value of the R code snipped embedded between the brackets:

In [5]:
[rprint| 1 + 1 |]

[1] 2

You can define the factorial function using R code, just as you can using Haskell code, so long as the R code is delineated within a quasiquote:

In [6]:
[r| fact <<- function(n) if(n == 0) 1 else n * fact(n - 1) |]

0x00007fb35852f398

The `r` quasiquote is used for embedding R code that is only useful for its side effects. This is the case with the code above which has the side effect of binding `fact` in the toplevel environment. Applying the definition:

In [7]:
[rprint| fact(10) |]

[1] 3628800

## R graphics

R has extremely powerful plotting facilities. They are available out-of-the-box:

In [8]:
[rgraph| plot(cars) |]

R Runtime Error: Error in (function (path_hs)  : could not find function "png"

For effect, we can fit a straight line through our data set:

In [9]:
[rgraph| plot(cars); abline(lm(cars$dist ~ cars$speed), col="red") |]

R Runtime Error: Error in (function (path_hs)  : could not find function "png"

R code snippets that have graphical output should be embedded using the `rgraph` quasiquote. The other quasiquotes ignore graphical output. For a more complex example, consider the following density plot (requires `ggplot2` to be installed):

In [10]:
[r| require("ggplot2") |]

0x00007fb3582c8d08

In [ ]:
[rgraph|
  Xv <- c(rnorm (500, 10,3), rnorm (500, 50, 20), rnorm (500, 70, 20))
  Yv <- c(rnorm (500, 10,3), rnorm (500, 70, 5), rnorm (500, 30, 5))
  myd <- data.frame(Xv, Yv)

  ggplot(myd, aes(x = Xv, y = Yv)) + geom_point() + geom_density2d() + theme_bw() |]